In [82]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import scipy.optimize
import numpy as np
from matplotlib import pyplot as plt
%matplotlib ipympl

In [2]:
response = requests.get("https://www.gov.uk/government/publications/coronavirus-covid-19-number-of-cases-in-england/coronavirus-covid-19-number-of-cases-in-england")

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
table = soup.select_one('.govspeak table')

In [5]:
rows = table.select('tr')

In [6]:
data = [tuple(c.text for c in r.select('td')) for r in rows[1:]]

In [7]:
df = pd.DataFrame(data, columns=['authority', 'cases'])
df['cases'] = pd.to_numeric(df['cases'])
df

,authority,cases
0,Barking and Dagenham,1
1,Barnet,8
2,Barnsley,2
3,Bath and North East Somerset,0
4,Bedford,0
...,...,...
145,Wokingham,3
146,Wolverhampton,3
147,Worcestershire,0
148,York,3


---

In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [14]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)

In [15]:
driver.get("https://www.arcgis.com/apps/opsdashboard/index.html#/f94c3c90da5b4e9f9a0b19484dd4bb14")

In [ ]:
//*[text()="Daily Confirmed Cases"]/ancestor::full-container[1]//div[@class="amcharts-chart-div"]//*[local-name() = 'svg']//*[local-name()='g'][@aria-label]

In [26]:
elements = driver.find_elements(By.XPATH, "//*[text()='Daily Confirmed Cases']/ancestor::full-container[1]//div[@class='amcharts-chart-div']//*[local-name() = 'svg']//*[local-name()='g'][@aria-label]")

In [31]:
attributes = [e.get_attribute('aria-label') for e in elements]

In [32]:
attributes[0]

'CMODateCount Jan 31, 2020 2'

In [56]:
dates_, counts_ = zip(*[re.match(r'CMODateCount (\w+ \d+, \d+) (\d+)', a).groups() for a in attributes])

In [57]:
dates = pd.to_datetime(dates_)
counts = pd.to_numeric(counts_)

In [58]:
df = pd.DataFrame({'dates': dates, 'new_cases': counts})
df['cases'] = df['new_cases'].cumsum()
df.tail()

,dates,new_cases,cases
35,2020-03-06,46,160
36,2020-03-07,46,206
37,2020-03-08,65,271
38,2020-03-09,50,321
39,2020-03-10,52,373


In [194]:
df.plot('dates', 'cases', kind='scatter', title='How Fucked We Are');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [195]:
def fit_func(x, a, m, c):
    return a * np.exp(m*x + c)

In [196]:
import matplotlib.dates as dates

In [197]:
days = df['dates'].dt.dayofyear
popt, pcov = scipy.optimize.curve_fit(fit_func, days, df['cases'])

In [198]:
plt.plot(df['dates'], fit_func(days, *popt), 'g');

In [218]:
t_projection = days.to_numpy()[-1] + np.arange(7)
dt_projection = pd.to_datetime(t_projection - 1, unit='D', origin=pd.Timestamp("2020-01-01"))

In [202]:
plt.plot(dt_projection, fit_func(t_projection, *popt), 'r--');